### Álvaro Felipe

La idea es usar una red convolucional ya entrenada para clasificar las imágenes, entrenaremos por tanto una red densa clasificadora, pero la visión por computador la realizará una CNN preentrenada. Vamos a hacer un flow from directory para cargar los datos de forma optima.

Hemos usado una vgg16 preentrenada con imagenet para clasificar nuestras fotos. disponemos de 21179 fotos de animales, en total tenemos 10 clases. La red vgg necesita de entrada fotos de tamaño 224x224 por tanto, como nuestras fotos son de tamaño y dimensiones variables, reescalamos antes de pasar la entrada a la red. 
Nuestra red densa de clasificación consta de 4 capas densas y 1 de dropout para evitar overfitting, las 3 primeras capas densas utilizan relu para facilitar la propagación de gradiente, mientras que la ultima consta de 10 neuronas (1 por cada clase) y una función de activación softmax (propia para las clasificaciones multiclase).


In [1]:
from keras.applications import VGG16
from keras.models import Sequential, load_model
from keras.layers import Flatten, Dense, Dropout 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import os

c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framewo

### entrenar


Hemos creado 2 funciones de entrenamiento, una para el entrenamiento inicial y otra para cargar un modelo medio entrenado y refinarlo.

In [ ]:
def entrenamiento(batch_size=20, epochs=20, num_samples=21.179):

    train_data_dir = 'train_images/train_images' # carpeta train
    tamano_entrada=(224,224)  #necesario para usar la vgg16

    # VGG16 preentrenada que evitamos que entrene y quitamos las capas densas de predicción
    base_model = VGG16(weights='imagenet', include_top=False,
                       input_shape=(*tamano_entrada, 3))
    base_model.trainable = False

    # clasificador
    model = Sequential()
    model.add(base_model)
    model.add(Flatten()) #para entrada a capas densas tiene q ser flatten
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))  # 10 clases

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])

    # generadores de datos desde flow from dir
    val_split=0.2
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       validation_split=val_split)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=tamano_entrada, 
        batch_size=batch_size,
        class_mode='categorical',  
        subset='training'            
    )

    validation_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=tamano_entrada,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )    

    # entrenar
    history = model.fit(
        train_generator,
        steps_per_epoch=num_samples*(1-val_split) // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=num_samples*val_split // batch_size,
        verbose=1
    )

    return history, model


In [ ]:
 # primer entrernamiento 
history, model = entrenamiento(epochs=6)

# --- Guardar modelo entrenado ---
model_path = "modelo_vgg16_1.h5"
model.save(model_path)
print(f"\n Modelo guardado en: {os.path.abspath(model_path)}")

In [2]:
# definimos esta funcion para seguir entrenando un modelo importado
def entrenamiento_precargado(batch_size=20, epochs=20, num_samples=21.179):

    train_data_dir = 'train_images/train_images'
    tamano_entrada=(224,224)  #necesario para usar la vgg16

    model=load_model("modelo_vgg16_1.h5")

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])

    # generadores de datos desde flow from dir
    val_split=0.2
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       validation_split=val_split)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=tamano_entrada, 
        batch_size=batch_size,
        class_mode='categorical',  
        subset='training'            
    )

    validation_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=tamano_entrada,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )    

    # entrenar
    history = model.fit(
        train_generator,
        steps_per_epoch=num_samples*(1-val_split) // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=num_samples*val_split // batch_size,
        verbose=1
    )

    return history, model


In [ ]:
# segundo entrenamiento
history, model = entrenamiento_precargado(epochs=4)

# --- Guardar modelo entrenado ---
model_path = "modelo_vgg16_2.h5"
model.save(model_path)
print(f"\n Modelo guardado en: {os.path.abspath(model_path)}")

Found 16947 images belonging to 10 classes.
Found 4232 images belonging to 10 classes.
Epoch 1/4


c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


848/848 ━━━━━━━━━━━━━━━━━━━━ 1122s 1s/step - accuracy: 0.9372 - loss: 0.1769 - val_accuracy: 0.8698 - val_loss: 0.4291
Epoch 2/4
848/848 ━━━━━━━━━━━━━━━━━━━━ 1115s 1s/step - accuracy: 0.9545 - loss: 0.1343 - val_accuracy: 0.8691 - val_loss: 0.4911
Epoch 3/4
848/848 ━━━━━━━━━━━━━━━━━━━━ 1167s 1s/step - accuracy: 0.9587 - loss: 0.1200 - val_accuracy: 0.8634 - val_loss: 0.4933
Epoch 4/4
653/848 ━━━━━━━━━━━━━━━━━━━━ 3:30 1s/step - accuracy: 0.9715 - loss: 0.0897

### predecir

In [ ]:
# mapeo de clases a sus respectivos nombres (realizado a mano porque se me olvidó guardar las clases antes de entrenar la primera vez).
labels = {
    0: "cane",
    1: "cavallo",
    2: "elefante",
    3: "farfalla",
    4: "gallina",
    5: "gatto",
    6: "mucca",
    7: "pecora",
    8: "ragno",
    9: "scoiattolo"
}

In [ ]:
import pandas as pd
import numpy as np
def predecir_test(model_path, batch_size=20):

    model = load_model(model_path)

    # Generador para test 
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_data_dir = "test_images/test_images"
    tamano_entrada=(224,224)  #necesario para usar la vgg16

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=tamano_entrada,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False      # mantener orden para construir el csv
    )

    # Predicciones
    preds = model.predict(test_generator, verbose=1)
    clases_pred = np.argmax(preds, axis=1)
    results = pd.DataFrame({
        "id": [os.path.basename(fname) for fname in test_generator.filenames],
        "category": [labels[idx] for idx in clases_pred]
    })

    return results


In [20]:
df_preds = predecir_test("modelo_vgg16_1.h5")
df_preds


Found 5000 images belonging to 1 classes.


c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 1190s 5s/step


,id,category
0,0.jpeg,scoiattolo
1,1.jpeg,cavallo
2,10.jpeg,scoiattolo
3,100.jpeg,pecora
4,1000.jpeg,cavallo
...,...,...
4995,995.jpeg,cavallo
4996,996.jpeg,mucca
4997,997.jpeg,cane
4998,998.jpeg,ragno


In [21]:
# Guardar 
df_preds.to_csv("predicciones_1.csv", index=False)